In [1]:
import os
from os import listdir
from os.path import join

from PIL import Image
import random

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import copy

import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import utils, models
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision
import torchvision.transforms as transforms

def set_seed(seed):

    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)

def is_image(filename):
    return any(filename.endswith(extension) for extension in [".jpg", ".jpeg", ".png"])
    
class LabeledDataset(Dataset):

    def __init__(self, dir_path, transform=None):
        """
        Args:
            dir_path (string): Directory containing the images.
            transform (optional): Optional transform to be applied
                on an image.
        """

        self.transform = transform
        
        # The list of all the image file names (but not the images themselves!) will be read
        # when the Dataset object is initialized
        self.image_filenames = [join(dir_path, f) for f in listdir(dir_path) if is_image(f)]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):

        img_name = self.image_filenames[idx]
        
        # Here is where the image actually gets read:
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)
            
        label = int(img_name[-5])

        return image, label
    
rotation = transforms.RandomChoice([transforms.RandomRotation([-3,3]),
                                    transforms.RandomRotation([87,93]), 
                                    transforms.RandomRotation([177,183]),
                                    transforms.RandomRotation([267, 273])])

augmentation = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(), 
                                   rotation])
    
labeled_set = LabeledDataset('data/MelanomaDetection/labeled',     
                             transform=transforms.Compose([augmentation, transforms.ToTensor()]))

set_seed(1)
train_set, val_set = torch.utils.data.random_split(labeled_set, [int(0.7*len(labeled_set)), int(0.3*len(labeled_set))])
 
test_set = LabeledDataset('data/MelanomaDetection/test', transform=transforms.ToTensor())  

print('Number of labeled TRAIN images: ', len(train_set))
print('Number of VAL images: ', len(val_set))
print('Number of TEST images: ', len(test_set))

train_loader = DataLoader(train_set, batch_size=len(train_set), shuffle=False, num_workers=0)
val_loader = DataLoader(val_set, batch_size=len(val_set), shuffle=False, num_workers=0)
test_loader = DataLoader(test_set, batch_size=len(test_set), shuffle=False, num_workers=0)

Number of labeled TRAIN images:  140
Number of VAL images:  60
Number of TEST images:  600


In [ ]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt

parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default = 'imagenet', help='cifar10 | lsun | mnist |imagenet | fake')
parser.add_argument('--dataroot', default = './data/MelanomaDetection1', help='unlabeled and/or train/test data merged')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=2)
parser.add_argument('--batchSize', type=int, default=64, help='input batch size')
parser.add_argument('--imageSize', type=int, default=64, help='the height / width of the input image to network')
parser.add_argument('--nz', type=int, default=100, help='size of the latent z vector')
parser.add_argument('--ngf', type=int, default=64)
parser.add_argument('--ndf', type=int, default=64)
parser.add_argument('--niter', type=int, default=200, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.0002, help='learning rate, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
parser.add_argument('--dry-run', action='store_true', help='check a single training cycle works')
parser.add_argument('--ngpu', type=int, default=1, help='number of GPUs to use')
parser.add_argument('--netG', default='', help="path to netG (to continue training)")
parser.add_argument('--netD', default='', help="path to netD (to continue training)")
parser.add_argument('--outf', default='./data/x', help='folder to output images and model checkpoints')
parser.add_argument('--classes', default='bedroom', help='comma separated list of classes for the lsun data set')

opt, unknown = parser.parse_known_args()


opt.manualSeed = random.randint(1, 10000)
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)

cudnn.benchmark = True


if opt.dataset in ['imagenet']:
    # folder dataset
    dataset = dset.ImageFolder(root=opt.dataroot,
                               transform=transforms.Compose([
                                   transforms.Resize(opt.imageSize),
                                   transforms.CenterCrop(opt.imageSize),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ]))
    nc=3

assert dataset
dataloader = torch.utils.data.DataLoader(dataset, batch_size=opt.batchSize,
                                         shuffle=True, num_workers=int(opt.workers))

device = torch.device("cuda:0")
ngpu = int(opt.ngpu)
nz = int(opt.nz)
ngf = int(opt.ngf)
ndf = int(opt.ndf)


# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.zeros_(m.bias)


class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output


netG = Generator(ngpu).to(device)
netG.apply(weights_init)
if opt.netG != '':
    netG.load_state_dict(torch.load(opt.netG))
print(netG)


class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)


netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
if opt.netD != '':
    netD.load_state_dict(torch.load(opt.netD))
print(netD)

criterion = nn.BCELoss()

fixed_noise = torch.randn(opt.batchSize, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []

if opt.dry_run:
    opt.niter = 1

for epoch in range(opt.niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label,
                           dtype=real_cpu.dtype, device=device)

        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step() 
        
        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, opt.niter, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

            
            

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, opt.niter, i, len(dataloader),
                 errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % opt.outf,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),
                    '%s/fake_samples_epoch_%03d.png' % (opt.outf, epoch),
                    normalize=True)

        if opt.dry_run:
            break
    # do checkpointing
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (opt.outf, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (opt.outf, epoch))
    
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

Random Seed:  3864
Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    

[6/200][3/11] Loss_D: 0.0001 Loss_G: 36.7550 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[6/200][4/11] Loss_D: 0.0004 Loss_G: 36.4371 D(x): 0.9996 D(G(z)): 0.0000 / 0.0000
[6/200][5/11] Loss_D: 0.0003 Loss_G: 36.2948 D(x): 0.9997 D(G(z)): 0.0000 / 0.0000
[6/200][6/11] Loss_D: 0.0002 Loss_G: 36.0647 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[6/200][7/11] Loss_D: 0.0001 Loss_G: 35.6032 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[6/200][8/11] Loss_D: 0.0004 Loss_G: 35.4296 D(x): 0.9996 D(G(z)): 0.0000 / 0.0000
[6/200][9/11] Loss_D: 0.0001 Loss_G: 34.8285 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[6/200][10/11] Loss_D: 0.0005 Loss_G: 33.9244 D(x): 0.9995 D(G(z)): 0.0000 / 0.0000
[7/200][0/11]	Loss_D: 0.0002	Loss_G: 32.7804	D(x): 0.9998	D(G(z)): 0.0000 / 0.0000
[7/200][0/11] Loss_D: 0.0002 Loss_G: 32.7804 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[7/200][1/11] Loss_D: 0.0002 Loss_G: 30.1445 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[7/200][2/11] Loss_D: 0.0001 Loss_G: 25.7479 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[7/

[14/200][7/11] Loss_D: 0.0002 Loss_G: 38.5724 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[14/200][8/11] Loss_D: 0.0013 Loss_G: 38.5775 D(x): 0.9988 D(G(z)): 0.0000 / 0.0000
[14/200][9/11] Loss_D: 0.0003 Loss_G: 38.5667 D(x): 0.9997 D(G(z)): 0.0000 / 0.0000
[14/200][10/11] Loss_D: 0.0001 Loss_G: 38.5971 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[15/200][0/11]	Loss_D: 0.0010	Loss_G: 38.4498	D(x): 0.9990	D(G(z)): 0.0000 / 0.0000
[15/200][0/11] Loss_D: 0.0010 Loss_G: 38.4498 D(x): 0.9990 D(G(z)): 0.0000 / 0.0000
[15/200][1/11] Loss_D: 0.0005 Loss_G: 38.5657 D(x): 0.9996 D(G(z)): 0.0000 / 0.0000
[15/200][2/11] Loss_D: 0.0001 Loss_G: 38.7619 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[15/200][3/11] Loss_D: 0.0001 Loss_G: 38.5379 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[15/200][4/11] Loss_D: 0.0001 Loss_G: 38.5380 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[15/200][5/11] Loss_D: 0.0002 Loss_G: 38.5448 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[15/200][6/11] Loss_D: 0.0008 Loss_G: 38.4670 D(x): 0.9992 D(G(z)): 0.0000 

[22/200][10/11] Loss_D: 0.0002 Loss_G: 38.1623 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[23/200][0/11]	Loss_D: 0.0000	Loss_G: 38.4888	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
[23/200][0/11] Loss_D: 0.0000 Loss_G: 38.4888 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[23/200][1/11] Loss_D: 0.0001 Loss_G: 38.3845 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[23/200][2/11] Loss_D: 0.0002 Loss_G: 38.3521 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[23/200][3/11] Loss_D: 0.0005 Loss_G: 38.4894 D(x): 0.9995 D(G(z)): 0.0000 / 0.0000
[23/200][4/11] Loss_D: 0.0000 Loss_G: 38.3349 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[23/200][5/11] Loss_D: 0.0002 Loss_G: 38.5889 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[23/200][6/11] Loss_D: 0.0000 Loss_G: 38.3288 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[23/200][7/11] Loss_D: 0.0006 Loss_G: 38.4818 D(x): 0.9994 D(G(z)): 0.0000 / 0.0000
[23/200][8/11] Loss_D: 0.0001 Loss_G: 38.2627 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[23/200][9/11] Loss_D: 0.0001 Loss_G: 38.3195 D(x): 0.9999 D(G(z)): 0.0000 

[31/200][1/11] Loss_D: 0.0001 Loss_G: 38.1036 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[31/200][2/11] Loss_D: 0.0001 Loss_G: 38.0366 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[31/200][3/11] Loss_D: 0.0001 Loss_G: 38.1804 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[31/200][4/11] Loss_D: 0.0002 Loss_G: 38.2840 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[31/200][5/11] Loss_D: 0.0000 Loss_G: 38.1955 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[31/200][6/11] Loss_D: 0.0000 Loss_G: 38.0662 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[31/200][7/11] Loss_D: 0.0000 Loss_G: 38.0109 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[31/200][8/11] Loss_D: 0.0002 Loss_G: 38.0497 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[31/200][9/11] Loss_D: 0.0000 Loss_G: 38.0375 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[31/200][10/11] Loss_D: 0.0002 Loss_G: 38.2341 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[32/200][0/11]	Loss_D: 0.0000	Loss_G: 38.1637	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
[32/200][0/11] Loss_D: 0.0000 Loss_G: 38.1637 D(x): 1.0000 D(G(z)): 0.0000 

[39/200][4/11] Loss_D: 0.0001 Loss_G: 37.9006 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[39/200][5/11] Loss_D: 0.0000 Loss_G: 37.9440 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[39/200][6/11] Loss_D: 0.0000 Loss_G: 38.0051 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[39/200][7/11] Loss_D: 0.0001 Loss_G: 37.8444 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[39/200][8/11] Loss_D: 0.0002 Loss_G: 37.9011 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[39/200][9/11] Loss_D: 0.0001 Loss_G: 37.9008 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[39/200][10/11] Loss_D: 0.0000 Loss_G: 38.0088 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[40/200][0/11]	Loss_D: 0.0001	Loss_G: 38.0378	D(x): 0.9999	D(G(z)): 0.0000 / 0.0000
[40/200][0/11] Loss_D: 0.0001 Loss_G: 38.0378 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[40/200][1/11] Loss_D: 0.0001 Loss_G: 38.0134 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[40/200][2/11] Loss_D: 0.0000 Loss_G: 37.9045 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[40/200][3/11] Loss_D: 0.0000 Loss_G: 37.8907 D(x): 1.0000 D(G(z)): 0.0000 

[47/200][7/11] Loss_D: 0.0000 Loss_G: 37.4105 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[47/200][8/11] Loss_D: 0.0001 Loss_G: 37.3969 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[47/200][9/11] Loss_D: 0.0001 Loss_G: 37.4715 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[47/200][10/11] Loss_D: 0.0000 Loss_G: 37.4736 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[48/200][0/11]	Loss_D: 0.0001	Loss_G: 37.4355	D(x): 0.9999	D(G(z)): 0.0000 / 0.0000
[48/200][0/11] Loss_D: 0.0001 Loss_G: 37.4355 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[48/200][1/11] Loss_D: 0.0000 Loss_G: 37.3040 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[48/200][2/11] Loss_D: 0.0001 Loss_G: 37.3819 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[48/200][3/11] Loss_D: 0.0001 Loss_G: 37.4315 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[48/200][4/11] Loss_D: 0.0000 Loss_G: 37.3842 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[48/200][5/11] Loss_D: 0.0000 Loss_G: 37.3477 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[48/200][6/11] Loss_D: 0.0001 Loss_G: 37.3540 D(x): 0.9999 D(G(z)): 0.0000 

[55/200][9/11] Loss_D: 0.0000 Loss_G: 36.3045 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[55/200][10/11] Loss_D: 0.0000 Loss_G: 36.3778 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[56/200][0/11]	Loss_D: 0.0000	Loss_G: 36.2375	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
[56/200][0/11] Loss_D: 0.0000 Loss_G: 36.2375 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[56/200][1/11] Loss_D: 0.0000 Loss_G: 36.3965 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[56/200][2/11] Loss_D: 0.0000 Loss_G: 36.2332 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[56/200][3/11] Loss_D: 0.0000 Loss_G: 36.0792 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[56/200][4/11] Loss_D: 0.0000 Loss_G: 36.1480 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[56/200][5/11] Loss_D: 0.0000 Loss_G: 36.1304 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[56/200][6/11] Loss_D: 0.0001 Loss_G: 36.0830 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[56/200][7/11] Loss_D: 0.0000 Loss_G: 35.9107 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[56/200][8/11] Loss_D: 0.0001 Loss_G: 35.9908 D(x): 0.9999 D(G(z)): 0.0000 

[64/200][0/11]	Loss_D: 0.2890	Loss_G: 5.1117	D(x): 0.9165	D(G(z)): 0.1695 / 0.0084
[64/200][0/11] Loss_D: 0.2890 Loss_G: 5.1117 D(x): 0.9165 D(G(z)): 0.1695 / 0.0084
[64/200][1/11] Loss_D: 0.3452 Loss_G: 3.3291 D(x): 0.7794 D(G(z)): 0.0615 / 0.0410
[64/200][2/11] Loss_D: 0.7357 Loss_G: 7.8607 D(x): 0.9376 D(G(z)): 0.4695 / 0.0007
[64/200][3/11] Loss_D: 1.1336 Loss_G: 3.0544 D(x): 0.4172 D(G(z)): 0.0083 / 0.0621
[64/200][4/11] Loss_D: 0.8068 Loss_G: 6.0837 D(x): 0.9636 D(G(z)): 0.4828 / 0.0037
[64/200][5/11] Loss_D: 0.2571 Loss_G: 5.3942 D(x): 0.8328 D(G(z)): 0.0489 / 0.0084
[64/200][6/11] Loss_D: 0.1593 Loss_G: 4.2694 D(x): 0.9328 D(G(z)): 0.0800 / 0.0200
[64/200][7/11] Loss_D: 0.4141 Loss_G: 5.4346 D(x): 0.9346 D(G(z)): 0.2580 / 0.0075
[64/200][8/11] Loss_D: 0.4127 Loss_G: 3.5468 D(x): 0.7299 D(G(z)): 0.0591 / 0.0400
[64/200][9/11] Loss_D: 0.4747 Loss_G: 5.8974 D(x): 0.9445 D(G(z)): 0.3194 / 0.0043
[64/200][10/11] Loss_D: 0.0830 Loss_G: 6.2127 D(x): 0.9593 D(G(z)): 0.0387 / 0.0040
[65

[72/200][4/11] Loss_D: 0.7816 Loss_G: 4.7480 D(x): 0.8003 D(G(z)): 0.3620 / 0.0121
[72/200][5/11] Loss_D: 0.7437 Loss_G: 2.0924 D(x): 0.6481 D(G(z)): 0.1257 / 0.1740
[72/200][6/11] Loss_D: 1.0216 Loss_G: 8.0304 D(x): 0.9567 D(G(z)): 0.5747 / 0.0006
[72/200][7/11] Loss_D: 1.7450 Loss_G: 2.9322 D(x): 0.3224 D(G(z)): 0.0047 / 0.0779
[72/200][8/11] Loss_D: 0.8008 Loss_G: 4.8134 D(x): 0.9619 D(G(z)): 0.4753 / 0.0133
[72/200][9/11] Loss_D: 0.5592 Loss_G: 3.7688 D(x): 0.7297 D(G(z)): 0.1470 / 0.0316
[72/200][10/11] Loss_D: 0.8530 Loss_G: 4.8531 D(x): 0.8224 D(G(z)): 0.3713 / 0.0186
[73/200][0/11]	Loss_D: 0.8693	Loss_G: 2.1419	D(x): 0.5825	D(G(z)): 0.1283 / 0.1573
[73/200][0/11] Loss_D: 0.8693 Loss_G: 2.1419 D(x): 0.5825 D(G(z)): 0.1283 / 0.1573
[73/200][1/11] Loss_D: 1.2898 Loss_G: 8.5171 D(x): 0.9333 D(G(z)): 0.6369 / 0.0011
[73/200][2/11] Loss_D: 2.0824 Loss_G: 3.7976 D(x): 0.2509 D(G(z)): 0.0059 / 0.0513
[73/200][3/11] Loss_D: 0.5257 Loss_G: 3.1717 D(x): 0.9339 D(G(z)): 0.3106 / 0.0809
[73